We are using the **schemaorg** python package to generate schema

More info: https://openschemas.github.io/schemaorg/

Github: https://www.github.com/openschemas/schemaorg

In [ ]:
# Import modules: xrray, schemaorg
import xarray
from schemaorg.main import Schema 
from schemaorg.main.parse import RecipeParser                # For schema validation
from schemaorg.templates.google import make_dataset          # To generate html


# Load data with metadata (Type: netcdf)
data = xarray.open_dataset('data/amgeo/01-01-2013-12-30-00.nc')


In [ ]:
## Create a 'Dataset' Schema object
SchemaObj = Schema("Dataset")

In [ ]:
## Add a single property
SchemaObj.add_property("name","AMGeO Assimilative Maps For 01/01/2013 12:30:00")

# Adding url to the landing page
SchemaObj.add_property("url","https://amgeo.colorado.edu/data/sampledata.html")


In [ ]:
## Add a list of values
keywords = ["polar ionosphere","magnetosphere-ionosphere coupling", "ionospheric electrodynamics"]
SchemaObj.add_property("keywords",keywords)

In [ ]:
## Add another schema object to the main obj

# Create an author schema object
author = Schema("Organization");
author.add_property("name","AMGeO")
author.add_property("email","amgeo@colorado.edu")
author.add_property("logo","https://amgeo.colorado.edu/static/img/amgeosmall.svg")

# Add author to parent schema
SchemaObj.add_property('author',author)

# Validation using recipe file


In [ ]:
## Validate a schema using a recipe

recipe = RecipeParser("recipe.yml")
recipe.validate(SchemaObj)


In [ ]:
# Adding Temporal Coverage property
dataTime = str(data.coords['time'].values[0])
dataTime = dataTime[:dataTime.index('.')]
SchemaObj.add_property("temporalCoverage",dataTime)

In [ ]:
# Adding Spatial Coverage property
minLat = min(data.coords['lat'].values)
minLon = min(data.coords['lon'].values)

maxLat = max(data.coords['lat'].values)
maxLon = max(data.coords['lon'].values)


Place = Schema("Place")
Geo = Schema("GeoShape")
Geo.add_property("box",f"{minLat} {minLon} {maxLat} {maxLon}")
Place.add_property("geo",Geo)

SchemaObj.add_property("spatialCoverage",Place)

In [ ]:
# Adding variable measured (Type: Property value)
variable_list =  list(data.keys())
            
variable_Schema = []
for variable in variable_list:
    variableS = Schema("PropertyValue")
    variableS.add_property("name",data.data_vars.get(variable).attrs['longname'])
    variableS.add_property("unitText",data.data_vars.get(variable).attrs['units'])
    variable_Schema.append(variableS)

SchemaObj.add_property("variableMeasured",variable_Schema)

In [ ]:
## Validate a schema using a recipe

recipe = RecipeParser("recipe.yml")
recipe.validate(SchemaObj)

# Generating ld-json and HTML

In [ ]:

# To generate only the json markup
json_markup = make_dataset(SchemaObj,template="google/dataset.html")
## Context should be https://schema.org
json_markup = json_markup.replace('"@context": "https://www.schema.org"','"@context": "https://schema.org/"')


# To generate HTML landing page with json markup embedded
html = make_dataset(SchemaObj,template="google/dataset-vue-table.html")
html = html.replace('"@context": "https://www.schema.org"','"@context": "https://schema.org/"')

print(json_markup)

In [ ]:
from IPython.core.display import display, HTML
HTML(html)